In [1]:
import nltk
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
# from keras.optimizers import SGD
import random
from sklearn.preprocessing import LabelEncoder

### Import and load the data file

In [2]:
with open('intents.json', 'r') as f:
    intents = json.load(f)

print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'Hello', 'Hey there', 'Hello restaurant Taiwan'], 'responses': ['Hello and welcome to restaurant Taiwan', 'Hey there', 'Hi']}, {'tag': 'order', 'patterns': ['I want food', 'I want to order food asap', 'Can you please take my order for food?', 'Take my order please', 'I want to place an order for Chinese food', 'I want to place an order', 'Would you please help me to order food?', 'Can you please order food for me'], 'responses': ['Sure, what would you like to order today?', 'Definitely, what would you like to have today?', "Certainly, I'll try to help you with that. What are you feeling like eating today?"]}, {'tag': 'opening_hours', 'patterns': ['what are the opening hours?', 'When is the restaurant closed?', 'Could you please give me the opening hours of the restaurant?', 'When is the restaurant open?'], 'responses': ['The restaurant is open from Thursday-Sunday from 5-10:30pm']}, {'tag': 'reservation', 'patterns': ['I want t

In [3]:
training_sentences = []
training_sentences_stem = []
training_labels = []
labels = []
documents = []
#responses = []

### Preprocessing

In [4]:
# functions

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, training_sentences, doc):
    
    tok_sens = []
    bag = []
    for w in tokenized_sentence:
        word = stem(w)
        tok_sens.append(word)
        
    for w in training_sentences:
        bag.append(1) if w in tok_sens else bag.append(0)
    print(bag)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[labels.index(doc[1])] = 1
    training.append([bag, output_row])
            
    return bag

In [5]:
for intent in intents['intents']:
    # add to our labels 
    training_label = intent['tag']
    training_labels.append(training_label)
    for pattern in intent['patterns']:
        # tokenize each word
        w = tokenize(pattern)
        training_sentences.extend(w)
    #responses.append(intent['responses'])
        # add document in the corpus
        documents.append((w, intent['tag']))
       
        


In [6]:
# print(responses)

In [7]:
print(documents)

[(['Hi'], 'greeting'), (['Hey'], 'greeting'), (['Hello'], 'greeting'), (['Hey', 'there'], 'greeting'), (['Hello', 'restaurant', 'Taiwan'], 'greeting'), (['I', 'want', 'food'], 'order'), (['I', 'want', 'to', 'order', 'food', 'asap'], 'order'), (['Can', 'you', 'please', 'take', 'my', 'order', 'for', 'food', '?'], 'order'), (['Take', 'my', 'order', 'please'], 'order'), (['I', 'want', 'to', 'place', 'an', 'order', 'for', 'Chinese', 'food'], 'order'), (['I', 'want', 'to', 'place', 'an', 'order'], 'order'), (['Would', 'you', 'please', 'help', 'me', 'to', 'order', 'food', '?'], 'order'), (['Can', 'you', 'please', 'order', 'food', 'for', 'me'], 'order'), (['what', 'are', 'the', 'opening', 'hours', '?'], 'opening_hours'), (['When', 'is', 'the', 'restaurant', 'closed', '?'], 'opening_hours'), (['Could', 'you', 'please', 'give', 'me', 'the', 'opening', 'hours', 'of', 'the', 'restaurant', '?'], 'opening_hours'), (['When', 'is', 'the', 'restaurant', 'open', '?'], 'opening_hours'), (['I', 'want', 't

In [8]:
ignore_words = ['?', '!', '.', ',']

for w in training_sentences:
    if w not in ignore_words:
        word = stem(w)
        training_sentences_stem.append(word)
    #    word = stem(w)
    #    training_sentences.append(word)
        
print(training_sentences_stem)

# version list comprehension
#ignore_words = ['?', '!', '.', ',']

#training_sentences = [stem(w) for w in training_sentences if w not in ignore_words]
#print(training_sentences)

['hi', 'hey', 'hello', 'hey', 'there', 'hello', 'restaur', 'taiwan', 'i', 'want', 'food', 'i', 'want', 'to', 'order', 'food', 'asap', 'can', 'you', 'pleas', 'take', 'my', 'order', 'for', 'food', 'take', 'my', 'order', 'pleas', 'i', 'want', 'to', 'place', 'an', 'order', 'for', 'chines', 'food', 'i', 'want', 'to', 'place', 'an', 'order', 'would', 'you', 'pleas', 'help', 'me', 'to', 'order', 'food', 'can', 'you', 'pleas', 'order', 'food', 'for', 'me', 'what', 'are', 'the', 'open', 'hour', 'when', 'is', 'the', 'restaur', 'close', 'could', 'you', 'pleas', 'give', 'me', 'the', 'open', 'hour', 'of', 'the', 'restaur', 'when', 'is', 'the', 'restaur', 'open', 'i', 'want', 'to', 'make', 'a', 'reserv', 'can', 'you', 'pleas', 'help', 'me', 'to', 'make', 'a', 'reserv', 'would', 'you', 'help', 'me', 'to', 'make', 'a', 'reserv', 'ok', 'wait', 'thank', 'you', 'well', 'thank', 'you', 'ok', 'thank', 'you', 'thank', 'you', 'so', 'much', 'thank', 'a', 'lot', 'thank', 'you']


In [9]:
print(training_labels)

['greeting', 'order', 'opening_hours', 'reservation', 'user_thanks']


In [10]:
# remove duplicates and sort it
training_sentences = sorted(set(training_sentences_stem))
labels = sorted(set(training_labels))
print(training_sentences)
print(labels)

['a', 'an', 'are', 'asap', 'can', 'chines', 'close', 'could', 'food', 'for', 'give', 'hello', 'help', 'hey', 'hi', 'hour', 'i', 'is', 'lot', 'make', 'me', 'much', 'my', 'of', 'ok', 'open', 'order', 'place', 'pleas', 'reserv', 'restaur', 'so', 'taiwan', 'take', 'thank', 'the', 'there', 'to', 'wait', 'want', 'well', 'what', 'when', 'would', 'you']
['greeting', 'opening_hours', 'order', 'reservation', 'user_thanks']


### Create training and test data

In [11]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(labels)
print(output_empty)

[0, 0, 0, 0, 0]


In [12]:
for doc in documents:
    pattern_sentence = doc[0]
    bag = bag_of_words(pattern_sentence, training_sentences, doc)

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
X_train = list(training[:,0])
y_train = list(training[:,1])
print("Training data created")

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,

<ipython-input-12-a132b71adee8>:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


### Build the model

In [13]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax

model = Sequential()
model.add(Dense(128, input_shape=(len(X_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',  metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(X_train), np.array(y_train), epochs=200, batch_size=5, verbose=1)
# model.save('chatbot_model.h5', hist)

print(model.summary())

Epoch 1/200
6/6 [==============================] - 0s 1ms/step - loss: 1.6170 - accuracy: 0.2308
Epoch 2/200
6/6 [==============================] - 0s 1ms/step - loss: 1.6261 - accuracy: 0.2308
Epoch 3/200
6/6 [==============================] - 0s 600us/step - loss: 1.6094 - accuracy: 0.1923
Epoch 4/200
6/6 [==============================] - 0s 600us/step - loss: 1.4344 - accuracy: 0.3462
Epoch 5/200
6/6 [==============================] - 0s 1ms/step - loss: 1.4119 - accuracy: 0.4231
Epoch 6/200
6/6 [==============================] - 0s 800us/step - loss: 1.4652 - accuracy: 0.3462
Epoch 7/200
6/6 [==============================] - 0s 800us/step - loss: 1.3269 - accuracy: 0.5385
Epoch 8/200
6/6 [==============================] - 0s 800us/step - loss: 1.3121 - accuracy: 0.3846
Epoch 9/200
6/6 [==============================] - 0s 600us/step - loss: 1.2461 - accuracy: 0.6154
Epoch 10/200
6/6 [==============================] - 0s 1ms/step - loss: 1.1801 - accuracy: 0.5769
Epoch 11/200
6/6 

6/6 [==============================] - 0s 600us/step - loss: 0.0571 - accuracy: 0.9615
Epoch 84/200
6/6 [==============================] - 0s 800us/step - loss: 0.0176 - accuracy: 1.0000
Epoch 85/200
6/6 [==============================] - 0s 801us/step - loss: 0.0231 - accuracy: 1.0000
Epoch 86/200
6/6 [==============================] - 0s 800us/step - loss: 0.0171 - accuracy: 1.0000
Epoch 87/200
6/6 [==============================] - 0s 801us/step - loss: 0.0244 - accuracy: 1.0000
Epoch 88/200
6/6 [==============================] - 0s 600us/step - loss: 0.0376 - accuracy: 1.0000
Epoch 89/200
6/6 [==============================] - 0s 800us/step - loss: 0.0035 - accuracy: 1.0000
Epoch 90/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0114 - accuracy: 1.0000
Epoch 91/200
6/6 [==============================] - 0s 600us/step - loss: 0.0153 - accuracy: 1.0000
Epoch 92/200
6/6 [==============================] - 0s 800us/step - loss: 0.0244 - accuracy: 1.0000
Epoch 93/200
6/

6/6 [==============================] - 0s 800us/step - loss: 0.0018 - accuracy: 1.0000
Epoch 164/200
6/6 [==============================] - 0s 600us/step - loss: 3.2469e-05 - accuracy: 1.0000
Epoch 165/200
6/6 [==============================] - 0s 800us/step - loss: 8.7188e-05 - accuracy: 1.0000
Epoch 166/200
6/6 [==============================] - 0s 800us/step - loss: 0.0013 - accuracy: 1.0000
Epoch 167/200
6/6 [==============================] - 0s 600us/step - loss: 0.0016 - accuracy: 1.0000
Epoch 168/200
6/6 [==============================] - 0s 800us/step - loss: 1.3130e-04 - accuracy: 1.0000
Epoch 169/200
6/6 [==============================] - 0s 1ms/step - loss: 5.2279e-04 - accuracy: 1.0000
Epoch 170/200
6/6 [==============================] - 0s 600us/step - loss: 1.5435e-04 - accuracy: 1.0000
Epoch 171/200
6/6 [==============================] - 0s 800us/step - loss: 0.0079 - accuracy: 1.0000
Epoch 172/200
6/6 [==============================] - 0s 600us/step - loss: 0.0014 - acc

### Predict the response

In [14]:
def clean_up_sentence(sentence):
    
    sentence_words_stem = []
    # tokenize the pattern
    sentence_words = tokenize(sentence)
    print(sentence_words)
    # stem each word
    
    for word in sentence_words:
        words = stem(word)
        sentence_words_stem.append(words)

    return sentence_words_stem

In [15]:
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [16]:
p = bow("Can you please take my order for food?", training_sentences)
print(p)
print(labels)

['Can', 'you', 'please', 'take', 'my', 'order', 'for', 'food', '?']
[0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 1]
['greeting', 'opening_hours', 'order', 'reservation', 'user_thanks']


In [17]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, training_sentences,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": labels[r[0]], "probability": str(r[1])})
    return return_list

In [18]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [19]:
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [28]:
ints = predict_class('I would like to have food', model)
print(ints)

['I', 'would', 'like', 'to', 'have', 'food']
[{'intent': 'order', 'probability': '0.9999988'}]


In [29]:
#ints = predict_class('ok waiting thank you', model)
#ints = return_list[0]['intent']
#print(ints)

In [30]:
getResponse(ints , intents )

"Certainly, I'll try to help you with that. What are you feeling like eating today?"

In [23]:
#Creating GUI with tkinter
import tkinter
from tkinter import *
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)
#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)
ChatLog.config(state=DISABLED)
#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set
#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )
#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)
#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)
base.mainloop()

['hi']
['can', 'i', 'place', 'an', 'order', '?']
['hello']
['hey', 'there']
['hello', 'taiwan']
['i', 'want', 'food']
['thank', 'you']
